# QuantumOptics.jl
[QuantumOptics.jl](https://qojulia.org/). A Julia Framework for Open Quantum Dynamics.

Project created by [Helmut Ritsch's CQED group](https://www.uibk.ac.at/th-physik/cqed/) (Cavity Quantum Electrodynamics) at the Institute for Theoretical Physics of the University of Innsbruck. Leaded by [Sebastian Krämer](https://github.com/bastikr).

Inspired by [Quantum Optics Toolbox](https://qo.phy.auckland.ac.nz/toolbox/) for MATLAB and it succesor [Quantum Toolbox in Python](http://qutip.org/), form now QuTiP. Written in Julia, new open source languge designed for scientific computing.

In [ ]:
using QuantumOptics, PyPlot

# 1. Julia overlook

## 1.1 Linear algebra

## Close to $+\infty$ starting time for everything. It is Julia tradition

In [ ]:
σ₁ = [0 1;
      1 0];
σ₂ = [0 -im;
      im 0];
σ₃ = [1  0;
      0 -1]

In [ ]:
σ₁ * σ₂

In [ ]:
σ₁^2

In [ ]:
n = 1_000

In [ ]:
1 / 2

In [ ]:
1 // 2

In [ ]:
1//2 + 1//4

## Unfortunetly
Unicode limits Julia (C#, Rust, etc.) abilitis to use super- and subscripts limiting it to 30 concret case. It should be at least 2 $\times$ bigger. By there is a hope, that Unicode will be updated, look [*Unicode Proposal to Encode Subscripts/Superscripts for Mathematical Programming*](https://github.com/stevengj/subsuper-proposal).

In [ ]:
v₁ = [1 2 0]
v₂ = [0 -2 3]

In [ ]:
v₁ + v₂

### Proper vector

In [ ]:
v₂ = [0, 1, 0]  # Proper vector

In [ ]:
v₁ * v₂

In [ ]:
v₁'

In [ ]:
v₂'v₂

In [ ]:
v₁'v₁

In [ ]:
using LinearAlgebra

In [ ]:
v₁ ⋅ v₁

$v_{ 2 } \times v_{ 2 }$

In [ ]:
?×

In [ ]:
A = [0 1;
    1 0]

In [ ]:
typeof(A)

In [ ]:
issymmetric(A)

In [ ]:
eigen(A)

In [ ]:
?eigen

In [ ]:
eigen_A = eigen(A)

In [ ]:
eigen_A.values

In [ ]:
eigen_A.vectors[:, 1]

## 1.2 Multiple dispatch (generic functions)

$f( x ) = 2 ( x + 1 )^{ 2 }$

In [ ]:
f(x) = 2(x + oneunit(x))^2

In [ ]:
f(0)

In [ ]:
f(-1)

In [ ]:
f(0.0)

In [ ]:
f(σ₁)

In [ ]:
@code_typed f(0)

In [ ]:
@code_typed f(0.0)

In [ ]:
@code_typed f(σ₁)

In [ ]:
@code_llvm f(0)

# 2. Squeezed Fock State

In [ ]:
b = FockBasis(50)

In [ ]:
?FockBasis

In [ ]:
a = destroy(b)
at = create(b)
H = 0.5(at^2 + a^2)

In [ ]:
ψ₀ = fockstate(b, 3)

In [ ]:
tout, ψₜ = timeevolution.schroedinger([0:0.25:1;], ψ₀, H)

In [ ]:
x = [-5:0.1:5;]

for i in 1:4
    subplot(2, 2, i)
    Q = qfunc(ψₜ[i], x, x)
    pcolor(x, x, Q)
end

tight_layout()

In [ ]:
?qfunc

### Again with different parameters

In [ ]:
b = FockBasis(50)

ψ₀ = fockstate(b, 10)
tout, ψₜ = timeevolution.schroedinger([0:0.25:1;], ψ₀, H)

x = [-12:0.1:12;]

for i in 1:4
    subplot(2, 2, i)
    Q = qfunc(ψₜ[i], x, x)
    pcolor(x, x, Q)
end

tight_layout()

In [ ]:
?fockstate

# 3. Spin-3/2 System

In [ ]:
SpinBasis(3/2)

In [ ]:
?SpinBasis

In [ ]:
3 / 2

In [ ]:
3//2

In [ ]:
b = SpinBasis(3//2)

In [ ]:
sm = sigmam(b)

In [ ]:
?sigmam

In [ ]:
H = 2 * sigmaz(b)

In [ ]:
J = [sm]
τ = [0:0.025:5;]
ω = [-5:0.05:25;]

In [ ]:
?dm

In [ ]:
?spinup

In [ ]:
spinup(b)
ρ₀ = dm(spinup(b))

In [ ]:
corr = timecorrelations.correlation(τ, ρ₀, H, J, sigmap(b), sm)
ω, S = timecorrelations.spectrum(ω, H, J, sm)

In [ ]:
subplot(2, 1, 1)
plot(τ, corr)
xlabel(L"\tau")
ylabel(L"\langle \sigma_+(\tau) \sigma_-(0)\rangle")
subplot(2, 1, 2)
plot(ω, S)
xlabel(L"\omega")
ylabel(L"S(\omega)")
tight_layout()

### Again with different parameters

In [ ]:
b = SpinBasis(1)
sm = sigmam(b)
H = 2*sigmaz(b)
J = [sm]
τ = [0:0.025:5;]
ω = [-5:0.05:25;]
ρ0 = dm(spinup(b))
corr = timecorrelations.correlation(τ, ρ0, H, J, sigmap(b), sm)
ω, S = timecorrelations.spectrum(ω, H, J, sm)

subplot(2, 1, 1)
plot(τ, corr)
xlabel(L"\tau")
ylabel(L"\langle \sigma_+(\tau) \sigma_-(0)\rangle")
subplot(2, 1, 2)
plot(ω, S)
xlabel(L"\omega")
ylabel(L"S(\omega)")
tight_layout()

savefig("spin.svg")

# 3. `N`-Level System

## `N` = 3

In [ ]:
b = NLevelBasis(3)

In [ ]:
?NLevelBasis

In [ ]:
t₁₂ = transition(b, 1, 2)

In [ ]:
t₂₃ = transition(b, 2, 3)
t₃₁ = transition(b, 1, 3)
H = 10 * (t₃₁ + dagger(t₃₁))
J = [1.2 * t₂₃, 0.6 * t₁₂]

In [ ]:
ψ₀ = basisstate(b, 1)

In [ ]:
T = [0:0.01:10;];

In [ ]:
tout, ψₜ = timeevolution.mcwf(T, ψ₀, H, J, seed=2)

In [ ]:
?timeevolution.mcwf

In [ ]:
plot(tout, expect(dm(basisstate(b, 3)), ψₜ), label=L"$3\rangle$")
plot(tout, expect(dm(basisstate(b, 2)), ψₜ), label=L"$|2\rangle$")
plot(tout, expect(dm(basisstate(b, 1)), ψₜ), label=L"$|1\rangle$")
xlabel("Time")
ylabel("Probability")
legend()
tight_layout()

savefig("nlevel.svg")

In [ ]:
b = NLevelBasis(4)
t12 = transition(b, 1, 2)
t23 = transition(b, 2, 3)
t31 = transition(b, 1, 3)
H = 10*(t31 + dagger(t31))
J = [1.2*t23, 0.6*t12]
psi0 = basisstate(b, 1)
T = [0:0.01:10;]
tout, psit = timeevolution.mcwf(T, psi0, H, J; seed=2)

plot(tout, expect(dm(basisstate(b, 4)), psit), label=L"$|4\rangle$")
plot(tout, expect(dm(basisstate(b, 3)), psit), label=L"$|3\rangle$")
plot(tout, expect(dm(basisstate(b, 2)), psit), label=L"$|2\rangle$")
plot(tout, expect(dm(basisstate(b, 1)), psit), label=L"$|1\rangle$")
xlabel("Time")
ylabel("Probability")
legend()
tight_layout()
savefig("4level.svg")

# 4. Jaynes-Cummings System

In [ ]:
b_spin = SpinBasis(1//2)
b_fock = FockBasis(200)
sp = sigmap(b_spin)
sm = sigmam(b_spin)
a = destroy(b_fock)
at = create(b_fock)

sp $\otimes$ a

In [ ]:
H = sp ⊗ a + sm ⊗ at

In [ ]:
?coherentstate

In [ ]:
T = [0:0.01:50;]
ψ₀ = spindown(b_spin) ⊗ coherentstate(b_fock, 6)

In [ ]:
tout, ψₜ = timeevolution.schroedinger(T, ψ₀, H)

In [ ]:
?timeevolution.schroedinger

In [ ]:
plot(tout, expect(1, sp * sm, ψₜ))
xlabel("Time")
ylabel("Spin excitation")
tight_layout()

### Again with different parameters

In [ ]:
b_spin = SpinBasis(1//2)
b_fock = FockBasis(200)
sp = sigmap(b_spin)
sm = sigmam(b_spin)
a = destroy(b_fock)
at = create(b_fock)
H = sp ⊗ a + sm ⊗ at
T = [0:0.01:50;]
ψ0 = spindown(b_spin) ⊗ coherentstate(b_fock, 6)
tout, ψₜ = timeevolution.schroedinger(T, ψ₀, H)

plot(tout, expect(1, sp * sm, ψₜ))
xlabel("Time")
ylabel("Spin excitation")
tight_layout()
savefig("composite.svg")

# 5. Closing remarks

# Opinion from authors of package

## Package in current state "can be useful and efficient and is already quite stable". The biggest >>disadvantage<< is that it is "not as feature-rich as we would like it to be".

# If you want help and write some code, you are welcome :). QO.jl needs you ;).

# If you use QO.jl, authors ask for citing this article

# Sebastian Krämer, David Plankensteiner, Laurin Ostermann, Helmut Ritsch, QuantumOptics.jl: A Julia framework for simulating open quantum systems, Computer Physics Communications, Volume 227, June 2018, Pages 109-116, https://doi.org/10.1016/j.cpc.2018.02.004.

# If you use Julia, creators of language ask you to cite this article

# Jeff Bezanson, Alan Edelman, Stefan Karpinski and Viral B. Shah Julia: A Fresh Approach to Numerical Computing,  (2017) SIAM Review, 59: 65–98. doi: 10.1137/141000671, url: http://julialang.org/publications/julia-fresh-approach-BEKS.pdf

# and add your paper to this list https://julialang.org/publications/.

# 6. QuantumInformation.jl

# Check it.

In [ ]:
using QuantumInformation

$| 1 \rangle$ for 2 dimensional system

In [ ]:
ket(1, 2)

In [ ]:
?ket

$| 2 \rangle$ for 2 dimensional system

In [ ]:
ket(2, 2)

In [ ]:
bra(1, 2)

In [ ]:
bra(1, 2) * ket(1, 2)

In [ ]:
ψ = (1 / √2) * (ket(1, 2) + ket(2, 2))

In [ ]:
ϕ = (1 / 2) * ket(1, 2) + (√3 / 2) * ket(2, 2)

In [ ]:
ϕ'ψ

In [ ]:
√(ϕ'ϕ)

In [ ]:
?ketbra  # Błąd <vakb| a nie <valb|

In [ ]:
ket(2, 4) * bra(3, 4)

In [ ]:
ket(2, 4) * ket(3, 4)'

$| \psi \rangle \langle \psi |$

In [ ]:
proj(ψ)

In [ ]:
ψ * ψ'

In [ ]:
max_entangled(4)

In [ ]:
max_mixed(4)

In [ ]:
werner_state(4, 0.4)

$\textrm{Tr}_{ A }( \rho_{ A } \otimes \sigma_{ B } )$

In [ ]:
ρ = [0.25 0.25im;
    -0.25im 0.75]

In [ ]:
σ = [0.4 0.1im; 
    -0.1im 0.6]

In [ ]:
ρ ⊗ σ

In [ ]:
ptrace(ρ ⊗ σ, [2, 2], [2])